In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv', index_col='PassengerId')        #make id-column the index column
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv', index_col='PassengerId')          #id is not a column anymore, instead acts as an index
submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/sample_submission.csv', index_col='PassengerId')

In [ ]:
train.head()
#train.shape
#test.shape
#submission.head()

In [ ]:
train.Survived.sum()
train.describe()

In [ ]:
print(train.isna().sum())
print("age missing %:", train.isna().sum()[4]/train.shape[0]*100)

### Imputation of Missing Values

In [ ]:
print(test.isna().sum())
from sklearn.impute import SimpleImputer

#imp = SimpleImputer(missing_values=np.nan, strategy='mean')
#imp.fit(test.Age)

In [ ]:
train=train.drop(columns=["Name", "Cabin", "Ticket"])
test=test.drop(columns=["Name", "Cabin", "Ticket"])
train = train.dropna()
#test= test.dropna()
train.shape

In [ ]:
test.isna().any()
test.isna().sum()

In [ ]:
test['Age']=test['Age'].fillna(test['Age'].mean())    
test['Fare']=test['Fare'].fillna(test['Fare'].mean())    
test.isna().sum()
test["Embarked"].value_counts(normalize="True")
#test["Embarked"]= test["Embarked"].isna() == "S"
test.isna().sum()
test["Embarked"].value_counts(normalize="True")
test["Embarked"]
test["Embarked"]=test["Embarked"].replace(np.NaN,"S")
test.isna().sum()
#test["Embarked"].value_counts(normalize="True")
len(test)

In [ ]:
print("Females survived:", len(train.loc[train.Sex == "female"][train.Survived == 1])/len(train))
print("Males survived:",len(train.loc[train.Sex == "male"][train.Survived == 1])/len(train))
print("Survival Rate:", len(train.loc[train.Survived ==1])/len(train))

In [ ]:
print("1st Class Survived", len(train.loc[train.Pclass == 1][train.Survived == 1])/len(train.loc[train.Pclass == 1]))
print("2nd Class Survived", len(train.loc[train.Pclass == 2][train.Survived == 1])/len(train.loc[train.Pclass == 2]))
print("3rd Class Survived", len(train.loc[train.Pclass == 3][train.Survived == 1])/len(train.loc[train.Pclass == 3]))

In [ ]:
print("Age under 5 survived", len(train.loc[train.Age <= 5][train.Survived == 1])/len(train.loc[train.Age <= 5]))

In [ ]:
len(train.loc[train.Age <= 5])/len(train)

In [ ]:
train.dtypes

In [ ]:
train.isna().any()
test.isna().any()

In [ ]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()

for c in train.columns:
    if train[c].dtype=='object': 
        enc = preprocessing.LabelEncoder()
        enc.fit(list(train[c].values) + list(test[c].values))   
        #print(list(le.classes_))
        train[c] = enc.transform(train[c].values)
        test[c] = enc.transform(test[c].values)

display(train.head())
display(test.head())

train.shape
test.shape

In [ ]:
#train=train.drop(columns=["Embarked"])
#test=test.drop(columns=["Embarked"])

In [ ]:
#train
train.reset_index(drop=True, inplace=True)
#test.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=["Survived"]), train['Survived'], random_state=1)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

fitted = model.fit(X_train, y_train)
preds = fitted.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(preds, y_test)

In [ ]:
clf = RandomForestClassifier()

clf.fit(train.drop(columns=["Survived"]), train['Survived'])
#submission['target'] = clf.predict_proba(test)[:, 1]
#submission.to_csv('random_forest.csv')

In [ ]:
submission.shape

In [ ]:
#submission['Survived'] = clf.predict_proba(test)[:, 1]
submission['Survived'] = clf.predict(test)

submission.to_csv('vanilla_rf_lbl_enc.csv')

In [ ]:
submission
len(submission)
submission.isna().any()
submission.shape
submission.value_counts()
#train["Survived"].value_counts()